<a href="https://colab.research.google.com/github/essieva-8/M.Tech/blob/main/Association_Rule_Mining_Apriori_and_FP_Growth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")


In [ ]:
df=pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Online Retail.xlsx',engine="openpyxl")
print("File loaded successfully!")

File loaded successfully!


In [ ]:
print(f"Initial shape: {df.shape}")


Initial shape: (541909, 8)


In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [ ]:
print("Columns:", df.columns.tolist())

Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


In [ ]:
# Select only 'InvoiceNo', 'Description' and 'Country' columns
df1 = df[['InvoiceNo', 'Description', 'Country']]
display(df1.head())

,InvoiceNo,Description,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
1,536365,WHITE METAL LANTERN,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,United Kingdom


Data Cleaning

In [ ]:
#Handling missing values
#Check for null values in each column
print("\nMissing Values Count")
df1.isnull().sum()


Missing Values Count


,0
InvoiceNo,0
Description,1454
Country,0


In [ ]:
df1.loc[171575]

,171575
InvoiceNo,551431
Description,NaN
Country,United Kingdom


In [ ]:
# Remove extra spaces from Description
df1["Description"] = df1["Description"].astype(str).str.strip()

In [ ]:
# Drop rows without an InvoiceNo
original_rows = df1.shape[0]
df1.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
print(f"Removed {original_rows - df1.shape[0]} rows with null InvoiceNo.")

Removed 0 rows with null InvoiceNo.


In [ ]:
# Remove "returns" (transactions with InvoiceNo starting with 'C')
df1['InvoiceNo'] = df1['InvoiceNo'].astype('str')
original_rows = df1.shape[0]
df1 = df1[~df1['InvoiceNo'].str.startswith('C')]
print(f"Removed {original_rows - df1.shape[0]} return transactions ('C' invoices).")

Removed 9288 return transactions ('C' invoices).


In [ ]:
# Remove non-product "items" like 'POSTAGE' or 'Manual'
original_rows = df1.shape[0]
df1 = df1[~df1['Description'].isin(['POSTAGE', 'Manual'])]
print(f"Removed {original_rows - df1.shape[0]} rows for 'POSTAGE' or 'Manual'.")

Removed 1454 rows for 'POSTAGE' or 'Manual'.


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 531167 entries, 0 to 541908
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   InvoiceNo    531167 non-null  object
 1   Description  531167 non-null  object
 2   Country      531167 non-null  object
dtypes: object(3)
memory usage: 16.2+ MB


In [ ]:
df1['InvoiceNo'].nunique()

21904

In [ ]:
df1.loc[279747]

,279747
InvoiceNo,561363
Description,FELTCRAFT PRINCESS CHARLOTTE DOLL
Country,United Kingdom


In [ ]:
df1['Country'].value_counts()

,count
Country,
United Kingdom,487287
Germany,8659
France,8102
EIRE,7889
Spain,2423
Netherlands,2326
Switzerland,1936
Belgium,1935
Portugal,1464


In [ ]:
# NOTE: The full dataset is heavily skewed by UK transactions.
# For more meaningful and faster results, it's common practice to
# analyze a single country. We'll filter for 'France'.
df1_fr = df1[df1['Country'] == 'France'].copy()
print(f"Filtered for 'France'. Working with {df1_fr.shape[0]} transactions.")

Filtered for 'France'. Working with 8102 transactions.


Transform Data into Transactional Format

In [ ]:
# We need to consolidate the items into a one-hot encoded format for France data.
# Rows = InvoiceNo, Columns = Description
# We only care if an item was bought (presence), not how many.

# Create a pivot table to get the one-hot encoded format for France data
# The values will indicate the presence (1) or absence (0) of an item in an invoice
basket_fr = (df1_fr.groupby(['InvoiceNo', 'Description'])['Description']
          .count().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

# Convert the counts to 0 (if not in transaction) or 1 (if in transaction)
def encode_units(x):
    if x > 0:
        return 1
    else:
        return 0

basket_fr_encoded = basket_fr.applymap(encode_units)
print(f"Transformation complete for France data. Final matrix shape: {basket_fr_encoded.shape}")
display(basket_fr_encoded.head())

Transformation complete for France data. Final matrix shape: (383, 1561)


Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Run Apriori Algorithm

In [ ]:
# Use the one-hot encoded basket for the France dataset
frequent_itemsets_apriori = apriori(basket_fr_encoded, min_support=0.01, use_colnames=True)

print("--- Frequent Itemsets (Apriori) ---")
frequent_itemsets_apriori.sort_values('support', ascending=False)

--- Frequent Itemsets (Apriori) ---


,support,itemsets
330,0.193211,(RABBIT NIGHT LIGHT)
369,0.185379,(RED TOADSTOOL LED NIGHT LIGHT)
319,0.174935,(PLASTERS IN TIN WOODLAND ANIMALS)
314,0.172324,(PLASTERS IN TIN CIRCUS PARADE)
399,0.161880,(ROUND SNACK BOXES SET OF4 WOODLAND)
...,...,...
29843,0.010444,"(ALARM CLOCK BAKELIKE PINK, LUNCH BOX I LOVE L..."
29842,0.010444,"(ALARM CLOCK BAKELIKE PINK, LUNCH BOX I LOVE L..."
29841,0.010444,"(ALARM CLOCK BAKELIKE PINK, LUNCH BOX I LOVE L..."
29840,0.010444,"(ALARM CLOCK BAKELIKE PINK, JUMBO BAG APPLES, ..."


Run FP-Growth Algorithm

In [ ]:
frequent_itemsets_fp = fpgrowth(basket_fr_encoded, min_support=0.01, use_colnames=True)

print("--- Frequent Itemsets (FP-Growth) ---")
frequent_itemsets_fp.sort_values('support', ascending=False)

--- Frequent Itemsets (FP-Growth) ---


,support,itemsets
410,0.193211,(RABBIT NIGHT LIGHT)
0,0.185379,(RED TOADSTOOL LED NIGHT LIGHT)
34,0.174935,(PLASTERS IN TIN WOODLAND ANIMALS)
145,0.172324,(PLASTERS IN TIN CIRCUS PARADE)
1,0.161880,(ROUND SNACK BOXES SET OF4 WOODLAND)
...,...,...
12452,0.010444,"(ROUND SNACK BOXES SET OF4 WOODLAND, LUNCH BAG..."
12453,0.010444,"(LUNCH BOX WITH CUTLERY RETROSPOT, ROUND SNACK..."
12454,0.010444,"(ALARM CLOCK BAKELIKE PINK, ROUND SNACK BOXES ..."
12455,0.010444,"(ROUND SNACK BOXES SET OF4 WOODLAND, SKULL LUN..."


In [ ]:
# Check if results are the same
# might be in a different order, but the content is identical
print(f"\nApriori found {len(frequent_itemsets_apriori)} itemsets.")
print(f"FP-Growth found {len(frequent_itemsets_fp)} itemsets.")


Apriori found 29860 itemsets.
FP-Growth found 29860 itemsets.


Generate and Interpret Association Rules

In [ ]:
# We'll use the frequent itemsets from FP-Growth.
# We are looking for rules with a 'lift' greater than 1.

print("\nGenerating association rules (metric='lift', min_threshold=1)...")
rules = association_rules(frequent_itemsets_fp, metric="lift", min_threshold=1)

# Sort the rules by 'lift' to see the strongest associations first
rules_sorted = rules.sort_values('lift', ascending=False)

print("\n--- Top Association Rules (Sorted by Lift) ---")
# Displaying the columns requested
print(rules_sorted[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10).to_string(index=False))

print("\n--- Interpretation Example (first rule) ---")
if not rules_sorted.empty:
    first_rule = rules_sorted.iloc[0]
    print(f"Rule: IF a customer buys {list(first_rule['antecedents'])},")
    print(f"THEN they are {first_rule['lift']:.2f} times MORE LIKELY")
    print(f"to also buy {list(first_rule['consequents'])}.")
    print(f"This rule has a confidence of {first_rule['confidence']:.0%}.")
else:
    print("No rules found with the current thresholds.")


Generating association rules (metric='lift', min_threshold=1)...

--- Top Association Rules (Sorted by Lift) ---
                                                                                                                 antecedents                                                                                                                                                                      consequents  support  confidence  lift
                                                 (LUNCH BOX I LOVE LONDON, LUNCH BAG APPLE DESIGN, ALARM CLOCK BAKELIKE RED) (JUMBO BAG APPLES, LUNCH BOX WITH CUTLERY RETROSPOT, ROUND SNACK BOXES SET OF4 WOODLAND, ALARM CLOCK BAKELIKE GREEN, SKULL LUNCH BOX WITH CUTLERY, CHILDRENS CUTLERY DOLLY GIRL) 0.010444         1.0 95.75
                                             (LUNCH BOX I LOVE LONDON, LUNCH BAG APPLE DESIGN, CHILDRENS CUTLERY DOLLY GIRL)     (JUMBO BAG APPLES, LUNCH BOX WITH CUTLERY RETROSPOT, ROUND SNACK BOXES SET OF4 WOODLAND, ALARM CLOC

In [ ]:
# Adjusting Parameters
# Let's try with different parameters
# We'll increase support to 0.05 (find more itemsets)
# and increase the lift threshold to 6 (find only very strong rules).

print("\n\n--- Running with Adjusted Parameters (min_support=0.05, min_lift=6) ---")

# 1. Re-run FP-Growth with new support
frequent_itemsets_adj = fpgrowth(basket_fr_encoded, min_support=0.05, use_colnames=True)
print(f"Found {len(frequent_itemsets_adj)} frequent itemsets with min_support=0.05")

# 2. Generate rules with new lift threshold
rules_adj = association_rules(frequent_itemsets_adj, metric="lift", min_threshold=6)
print(f"Found {len(rules_adj)} rules with min_lift > 6")

# 3. Display new rules
print("\n--- Top Adjusted Rules (Sorted by Lift) ---")
print(rules_adj.sort_values('lift', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head(10).to_string(index=False))

print("\n\nProcess complete.")



--- Running with Adjusted Parameters (min_support=0.05, min_lift=6) ---
Found 113 frequent itemsets with min_support=0.05
Found 22 rules with min_lift > 6

--- Top Adjusted Rules (Sorted by Lift) ---
                                            antecedents                                             consequents  support  confidence      lift
                         (PACK OF 6 SKULL PAPER PLATES)                            (PACK OF 6 SKULL PAPER CUPS) 0.052219    0.909091 13.927273
                           (PACK OF 6 SKULL PAPER CUPS)                          (PACK OF 6 SKULL PAPER PLATES) 0.052219    0.800000 13.927273
                         (CHILDRENS CUTLERY DOLLY GIRL)                            (CHILDRENS CUTLERY SPACEBOY) 0.065274    0.892857 12.665344
                           (CHILDRENS CUTLERY SPACEBOY)                          (CHILDRENS CUTLERY DOLLY GIRL) 0.065274    0.925926 12.665344
                             (ALARM CLOCK BAKELIKE RED) (ALARM CLOCK BAKELIKE PINK,